In [1]:
%load_ext autotime
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
# Import csv file to pandas dataframe and preview
df = pd.read_csv("gwq10.csv")
df.head()

,well_id,results,chemical,date,units,qualifer,rl,latitude,longitude,well_type,source
0,TULE-01,1.000,"1,1,1-Trichloroethane",11/29/2005,UG/L,=,0.002,35.883148,-119.269694,MUNICIPAL,USGS
1,TULE-02,0.003,"1,1,1-Trichloroethane",11/30/2005,UG/L,<,0.003,36.095139,-119.198500,MUNICIPAL,USGS
2,TULE-03,5.000,"1,1,1-Trichloroethane",11/30/2005,UG/L,NaN,0.002,36.108833,-119.322000,MUNICIPAL,USGS
3,TULE-04,0.003,"1,1,1-Trichloroethane",12/5/2005,UG/L,<,0.003,35.794139,-119.111389,MUNICIPAL,USGS
4,TULE-05,0.003,"1,1,1-Trichloroethane",12/5/2005,UG/L,<,0.003,35.889944,-119.046056,MUNICIPAL,USGS


time: 38.9 ms


In [3]:
# Check number of rows in dataframe
len(df)

373

time: 1.99 ms


In [4]:
# Convert well_id data type to string
df['well_id'] = df['well_id'].astype(str)
# Convert latitude and longitude data type to float
df['latitude'] = df['latitude'].astype(float)
df['longitude'] = df['longitude'].astype(float)

time: 2.99 ms


In [5]:
# Reverse geocode one record of lat/lon data to test code and determine county data
geolocator = Nominatim(user_agent="gwq10.py", timeout=3) # 3 second delay to timeout

# Create county column
county_column = []
# Loop through dataframe 
for i in range(len(df)):
    # obtain lat and lon for each row in data frame
    lat = df.latitude.iloc[i]
    lon = df.longitude.iloc[i]
    # use rate limiter to avoid timeout error
    reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1) # nominatum usage policy requires at least 1 second
    
    # get the address data for dataframe using lat/lon
    location = geolocator.reverse(f"{lat}, {lon}")
    # split address data into string
    geo_string = location.address.split(",")
    # extract county data from address data
    for i in geo_string:
        if 'County' in i:
            county_column.append(i)

time: 3min 36s


In [6]:
county_column[:10]

[' Tulare County',
 ' Tulare County',
 ' Tulare County',
 ' Tulare County',
 ' Tulare County',
 'Tulare County',
 ' Tulare County',
 ' Tulare County',
 ' Tulare County',
 ' Tulare County']

time: 2.99 ms


In [7]:
# add "county_column" data to dataframe and name "county"
df["county"] = pd.Series(county_column)

time: 2.99 ms


In [8]:
df.head(10)

,well_id,results,chemical,date,units,qualifer,rl,latitude,longitude,well_type,source,county
0,TULE-01,1.000,"1,1,1-Trichloroethane",11/29/2005,UG/L,=,0.002,35.883148,-119.269694,MUNICIPAL,USGS,Tulare County
1,TULE-02,0.003,"1,1,1-Trichloroethane",11/30/2005,UG/L,<,0.003,36.095139,-119.198500,MUNICIPAL,USGS,Tulare County
2,TULE-03,5.000,"1,1,1-Trichloroethane",11/30/2005,UG/L,NaN,0.002,36.108833,-119.322000,MUNICIPAL,USGS,Tulare County
3,TULE-04,0.003,"1,1,1-Trichloroethane",12/5/2005,UG/L,<,0.003,35.794139,-119.111389,MUNICIPAL,USGS,Tulare County
4,TULE-05,0.003,"1,1,1-Trichloroethane",12/5/2005,UG/L,<,0.003,35.889944,-119.046056,MUNICIPAL,USGS,Tulare County
5,TULE-06,0.003,"1,1,1-Trichloroethane",12/5/2005,UG/L,<,0.003,35.913611,-119.078611,MUNICIPAL,USGS,Tulare County
6,TULE-07,6.000,"1,1,1-Trichloroethane",12/6/2005,UG/L,NaN,0.002,36.058917,-119.307639,MUNICIPAL,USGS,Tulare County
7,TULE-08,7.000,"1,1,1-Trichloroethane",12/7/2005,UG/L,NaN,0.002,35.969556,-119.283778,MUNICIPAL,USGS,Tulare County
8,TULE-09,0.003,"1,1,1-Trichloroethane",12/7/2005,UG/L,<,0.003,36.061556,-119.028222,MUNICIPAL,USGS,Tulare County
9,TULE-10,0.003,"1,1,1-Trichloroethane",12/7/2005,UG/L,<,0.003,36.022500,-119.097167,MUNICIPAL,USGS,Tulare County


time: 53.9 ms


In [9]:
# make csv from df_dup dataframe
df.to_csv("gwq10-rgeocode.csv", index=None, header=True)

time: 17 ms
